This tutorial will guide you through the process of using _transfer learning_ to learn an accurate image classifier from a small number of training samples. Transfer learning refers to the process of taking an existing neural network which was previously trained to good performance on a larger dataset, and using it as the basis for a new model which leverages the previous model's accuracy for the new task. Another name for this procedure is called "fine-tuning" because we will take a previous neural network and fine-tune its weights to a new dataset.

How it works: we will first load a previously-trained neural net, Keras's built-in VGG16 model which was trained to do ImageNet classification on 1000 classes, and remove its final layer, the 1000-neuron softmax classification layer, and replace it with a new classification layer for the number of classes we are training on. We then "freeze" all the weights in the network except the ones connecting to the new classification layer, and then re-train the model on our new dataset. By keeping the earlier weights fixed, we get to use the features that were discoverd in the previous model.

We will compare using this method to training a small neural network from scratch on the new dataset, and as we shall see, it will dramatically improve our accuracy.

#http://stackoverflow.com/questions/41378461/how-to-use-models-from-keras-applications-for-transfer-learnig/41386444#41386444

In [12]:
import os
#os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"
#os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu"


import random
import numpy as np
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model
from tqdm import tqdm

In [2]:
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [3]:

root = '/Users/gene/Teaching/ML4A/ml4a-guides/data/101_ObjectCategories'
exclude = ['BACKGROUND_Google', 'Motorbikes', 'airplanes', 'Faces_easy', 'Faces']


data = []
categories = [x[0] for x in os.walk(root) if x[0]][1:]
categories = [c for c in categories if c not in [os.path.join(root, e) for e in exclude]]
for c, category in tqdm(enumerate(categories)):
    images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(os.path.join(root,category)) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':c})

num_classes = len(categories)

random.shuffle(data)
# print(len(data))
# data = data[:2400]

# create training / validation / test split (80%, 10%, 10%)
train = data[:int(0.8 * len(data))]
val = data[int(0.8 * len(data)):int(0.9 * len(data))]
test = data[int(0.9 * len(data)):]

# pre-processing
x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train]
x_val, y_val = np.array([t["x"] for t in val]), [t["y"] for t in val]
x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test]

y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_val /= 255
x_test /= 255


97it [00:24,  4.42it/s]


In [4]:
print(len(data))

6209


In [5]:

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 109, 109, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 32)        0         
__________

In [6]:
model.fit(x_train, y_train,
          batch_size=10,
          epochs=25,
          validation_data=(x_val, y_val))

Train on 4967 samples, validate on 621 samples
Epoch 1/25
4967/4967 [==============================] - 253s - loss: 4.5058 - acc: 0.0443 - val_loss: 4.4397 - val_acc: 0.0483
Epoch 2/25
4967/4967 [==============================] - 250s - loss: 4.2734 - acc: 0.0904 - val_loss: 4.0860 - val_acc: 0.0982
Epoch 3/25
4967/4967 [==============================] - 250s - loss: 3.9778 - acc: 0.1363 - val_loss: 3.9426 - val_acc: 0.1562
Epoch 4/25
4967/4967 [==============================] - 251s - loss: 3.7718 - acc: 0.1617 - val_loss: 3.7194 - val_acc: 0.1852
Epoch 5/25
4967/4967 [==============================] - 255s - loss: 3.6211 - acc: 0.1884 - val_loss: 3.5508 - val_acc: 0.1932
Epoch 6/25
4967/4967 [==============================] - 256s - loss: 3.4709 - acc: 0.2170 - val_loss: 3.3751 - val_acc: 0.2254
Epoch 7/25
4967/4967 [==============================] - 258s - loss: 3.3422 - acc: 0.2305 - val_loss: 3.2935 - val_acc: 0.2560
Epoch 8/25
4967/4967 [==============================] - 260s - l

MemoryError: Error allocating 63083520 bytes of device memory (out of memory).
Apply node that caused the error: GpuDnnPoolGrad{mode='max'}(GpuContiguous.0, GpuContiguous.0, GpuContiguous.0, TensorConstant{(2,) of 2}, TensorConstant{(2,) of 2}, TensorConstant{(2,) of 0})
Toposort index: 321
Inputs types: [CudaNdarrayType(float32, 4D), CudaNdarrayType(float32, 4D), CudaNdarrayType(float32, 4D), TensorType(int64, vector), TensorType(int64, vector), TensorType(int64, vector)]
Inputs shapes: [(10, 32, 222, 222), (10, 32, 111, 111), (10, 32, 111, 111), (2,), (2,), (2,)]
Inputs strides: [(1577088, 49284, 222, 1), (394272, 12321, 111, 1), (394272, 12321, 111, 1), (8,), (8,), (8,)]
Inputs values: ['not shown', 'not shown', 'not shown', array([2, 2]), array([2, 2]), array([0, 0])]
Outputs clients: [[GpuDimShuffle{0,2,3,1}(GpuDnnPoolGrad{mode='max'}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by se

In [7]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

('Test loss:', 2.7593512197239578)
('Test accuracy:', 0.38164251210129013)


In [10]:
model = None # get memory back
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [ ]:

frozen_model = Model(x, y)
# in the model below, the weights of `layer` will not be updated during training
frozen_model.compile(optimizer='rmsprop', loss='mse')

layer.trainable = True
trainable_model = Model(x, y)
# with this model the weights of the layer will be updated during training
# (which will also affect the above model since it uses the same layer instance)
trainable_model.compile(optimizer='rmsprop', loss='mse')

frozen_model.fit(data, labels)  # this does NOT update the weights of `layer`
trainable_model.fit(data, labels)  # this updates the weights of `layer`


In [27]:
new_classification_layer = Dense(num_classes, activation='softmax')
inp = vgg.input
out = new_classification_layer(vgg.layers[-2].output)
model_new = Model(inp, out)
model_new.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [32]:
for l, layer in enumerate(model2.layers[:-1]):
    layer.trainable = False

model2.compile(loss='categorical_crossentropy', 
               optimizer='adadelta', 
               metrics=['accuracy'])

In [34]:
model2.fit(x_train, y_train, 
           batch_size=8, 
           epochs=25, 
           validation_data=(x_val, y_val))

Train on 4967 samples, validate on 621 samples
Epoch 1/25
4967/4967 [==============================] - 1069s - loss: 2.7061 - acc: 0.3884 - val_loss: 2.1648 - val_acc: 0.4815
Epoch 2/25
4967/4967 [==============================] - 1057s - loss: 1.8858 - acc: 0.5617 - val_loss: 1.7625 - val_acc: 0.5684
Epoch 3/25
4967/4967 [==============================] - 1040s - loss: 1.5262 - acc: 0.6360 - val_loss: 1.5352 - val_acc: 0.6103
Epoch 4/25
4967/4967 [==============================] - 972s - loss: 1.3098 - acc: 0.6841 - val_loss: 1.3323 - val_acc: 0.6667
Epoch 5/25
4967/4967 [==============================] - 932s - loss: 1.1702 - acc: 0.7119 - val_loss: 1.2521 - val_acc: 0.6779
Epoch 6/25
4967/4967 [==============================] - 920s - loss: 1.0642 - acc: 0.7413 - val_loss: 1.1989 - val_acc: 0.6844
Epoch 7/25
4967/4967 [==============================] - 915s - loss: 0.9869 - acc: 0.7524 - val_loss: 1.1695 - val_acc: 0.7021
Epoch 8/25
4967/4967 [==============================] - 901s 

In [50]:
#score = model2.evaluate(x_test, y_test, verbose=0)

#print(len(x_test))
accs = []
sc = []
for i in range(124):
    print(i)
    score = model2.evaluate(x_test[i*5:(i+1)*5], y_test[i*5:(i+1)*5], verbose=0)
    sc.append(score[0])
    accs.append(score[1])
#
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123


In [51]:
accc = np.mean(accs)
scc = np.mean(sc)
print(accc,scc)

(0.75322581839657599, 0.94274442515245849)


In [52]:
score = model2.evaluate(x_test[0:5], y_test[0:5], verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

('Test loss:', 0.77414631843566895)
('Test accuracy:', 0.80000001192092896)


In [ ]:
model3 = Sequential()
model3.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=model.input_shape[1:]))
model3.add(Conv2D(64, (3, 3), activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.25))
model3.add(Flatten())
model3.add(Dense(128, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(num_classes, activation='softmax'))


model3.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

print(model3.summary())
model3.fit(x_train, y_train,
          batch_size=64,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model3.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#yy = model3.predict(x_test)
#print(yy)

score = model2.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

score = model3.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

score = model2.evaluate(x_train, y_train, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

score = model3.evaluate(x_train, y_train, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

